<a href="https://colab.research.google.com/github/enVives/TFG/blob/main/AnalisiDocDistancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import ast

from glob import glob
import json
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import drive


import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


import ast
from collections import Counter

drive.mount('/content/drive')

MODELS = {
    0: 'AlexnetBinary',
    1: 'Resnet152Binary',
    2: 'Inceptionv3Binary',
    3: 'EfficientNetB1Binary',
    4: 'AlexnetMulticlass',
    5: 'Resnet152Multiclass',
    6: 'Inceptionv3Multiclass',
    7: 'EfficientnetB1Multiclass',
}

DISTANCES = {
    'Cosin' : 0,
    'L2' : 1
}

MODEL = MODELS[6] #we pick a model
picked = 'Cosin'
DISTANCE = DISTANCES[picked] #we pick a distance

RUN = 9 #we pick a run

metadates_cos = pd.read_csv("/content/drive/MyDrive/Runs/Ham10000-"+MODEL+"/"+str(RUN)+"/Dist/"+picked+"/distances.txt", sep=";")
metadates_l2 = pd.read_csv("/content/drive/MyDrive/Runs/Ham10000-"+MODEL+"/"+str(RUN)+"/Dist/"+picked+"/distances.txt", sep=";")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
def process_and_get_mode(row):
    real_list = ast.literal_eval(row.replace(" ", ","))  # Convertir string a lista
    counter = Counter(real_list)
    return counter.most_common(1)[0][0]  # Devolver la moda

# x_cosin = x_cosin.apply(process_and_get_mode)
# x_l2 = x_l2.apply(process_and_get_mode)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title):
    # Crear la matriz de confusión
    cm = confusion_matrix(y_true, y_pred)

    # Visualizar con Seaborn
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=set(y_true), yticklabels=set(y_true))
    plt.xlabel("Etiqueta Majoritaria Dists")
    plt.ylabel("Test Image Label")
    plt.title(title)
    plt.show()

In [ ]:
dict_illnesses = {'nv': 0, 'mel': 1, 'bkl': 2, 'bcc': 3, 'akiec': 4, 'vasc': 5, 'df': 6}


# 1. Extract and drop index 2 from both Series
y_cosin = metadates_cos["test_img_label"].drop(index=2)
x_cosin = metadates_cos["top_images_labels"].drop(index=2)

x_real = []

for i in x_cosin:
    moda = process_and_get_mode(i)
    x_real.append(dict_illnesses[moda])

# 2. Build confusion matrix via crosstab
cosin_conf_matrix = pd.crosstab(
    y_cosin,
    x_real,
    rownames=["Actual"],
    colnames=["Predicted"],
    dropna=False  # include any labels that might be missing in one set
)

inv_dict_illnesses = {v: k for k, v in dict_illnesses.items()}
cosin_conf_matrix.index = cosin_conf_matrix.index.map(inv_dict_illnesses)
cosin_conf_matrix.columns = cosin_conf_matrix.columns.map(inv_dict_illnesses)

plt.figure(figsize=(10, 8))
sns.heatmap(cosin_conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix (Cosine Similarity)")
plt.ylabel("Classe imatge test")
plt.xlabel("Moda dels 5 més propers")
plt.show()

In [ ]:
y_labels = list(y_cosin)

# Comparar valores numéricos
a = 0
for i in range(len(x_real)):
    if x_real[i] != y_labels[i]:
        a += 1

error_rate = a / len(x_real)
print("Error rate:", error_rate)
print("Accuracy:", 1 - error_rate)

In [ ]:
dict_illnesses = {'nv': 0, 'mel': 1, 'bkl': 2, 'bcc': 3, 'akiec': 4, 'vasc': 5, 'df': 6}


# 1. Extract and drop index 2 from both Series
y_cosin = metadates_l2["test_img_label"].drop(index=2)
x_cosin = metadates_l2["top_images_labels"].drop(index=2)

x_real = []

for i in x_cosin:
    moda = process_and_get_mode(i)
    x_real.append(dict_illnesses[moda])

# 2. Build confusion matrix via crosstab
cosin_conf_matrix = pd.crosstab(
    y_cosin,
    x_real,
    rownames=["Actual"],
    colnames=["Predicted"],
    dropna=False  # include any labels that might be missing in one set
)

inv_dict_illnesses = {v: k for k, v in dict_illnesses.items()}
cosin_conf_matrix.index = cosin_conf_matrix.index.map(inv_dict_illnesses)
cosin_conf_matrix.columns = cosin_conf_matrix.columns.map(inv_dict_illnesses)

plt.figure(figsize=(10, 8))
sns.heatmap(cosin_conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix (L2 Similarity)")
plt.ylabel("Classe imatge test")
plt.xlabel("Moda dels 5 més propers")
plt.show()

In [ ]:
y_labels = list(y_cosin)

# Comparar valores numéricos
a = 0
for i in range(len(x_real)):
    if x_real[i] != y_labels[i]:
        a += 1

error_rate = a / len(x_real)
print("Error rate:", error_rate)
print("Accuracy:", 1 - error_rate)